In [ ]:
import json
import msgpack
import logbook

In [ ]:
exp_name = "17.1_coeditor_stats"
project_file = 'data/projects-2016-10-14-dedup.json'
edges_file = "archive/17_create_coeditor/2017-04-26 14:53:15 1b4e2e0/%d-coeditor.mp"

In [ ]:
exp = logbook.Experiment(exp_name)
log = exp.get_logger()
project_names = {}
with open(project_file, "rb") as f:
    for row in f:
        datum = json.loads(row)
        project_names[datum["project_id"]] = datum["project_name"]

In [ ]:
with open(exp.get_filename("coeditor_stats.tsv"), "wb") as out:
    out.write("project_id\tnodes\tedges\n")
    for project_id, project_name in project_names.iteritems():
        all_nodes = set()
        edge_count = 0
        project_id = 500
        edges_from = {}
        log.info("Loading network edges for project %d " % (project_id,))
        with open(edges_file % project_id, "rb") as f:
            unpacker = msgpack.Unpacker(f)
            for o in unpacker:
                edge_count += len(o[1])
                edges_from[o[0][0]] = set(o[1])
                all_nodes.add(o[0][0])
                all_nodes |= set(o[1])
        all_nodes = list(all_nodes)
        log.info("  Loaded %d nodes and %d edges" % (len(all_nodes), edge_count))
        out.write("%d\t%d\t%d\n" % (project_id, len(all_nodes), edge_count))